In [1]:
import psycopg2 as ppg
import pandas as pd
import numpy as np
import random
import datetime
import time



In [2]:
con = ppg.connect(dbname="tweets", user="patrick")

In [3]:
con.rollback()
with con.cursor() as cur:
    cur.execute("select user_id, max(tweets.created_at) as tweet_time, "
                "usr.created_at, last_tweet_at, (last_tweet_at < '2015-05-15') as churn, "
                "usr.followers_count, usr.friends_count from tweets inner join "
                "(select user_id, name, followers_count, friends_count, last_tweet_at, created_at from users "
                "where last_tweet_at is not null) as usr using (user_id) "
                "where tweets.created_at < '2015-04-04' group by user_id, usr.created_at, "
                "usr.last_tweet_at, usr.followers_count, usr.friends_count order by usr.created_at;")
    churn_df = pd.DataFrame(cur.fetchall(), columns=[c[0] for c in cur.description])

In [4]:
churn_df.columns
print churn_df.shape

(201809, 7)


In [5]:
# plot 
# training-testing
query_date = pd.to_datetime(datetime.datetime.strptime("2015-06-17", "%Y-%m-%d"))
churn_df['user_age'] = (query_date - churn_df.created_at).astype(np.int64)
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

training, testing = train_test_split(churn_df[['churn','followers_count','friends_count','user_age']], 
                                     train_size=0.8)

dscl = StandardScaler()
training_predictors = dscl.fit_transform(training.iloc[:,1:])
testing_predictors = dscl.transform(testing.iloc[:,1:])

/home/patrick/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:498: UserWarning: StandardScaler assumes floating point values as input, got int64
  "got %s" % (estimator, X.dtype))


In [23]:
training_predictors.shape

(161447, 3)

In [ ]:
# model

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.externals import joblib
import sklearn



def try_model(training_predictors, training_response, testing_predictors, testing_response, the_model):
    the_model.fit(training_predictors, training_response)
    classfn = the_model.predict(testing_predictors)
    print "Number correct: ", (classfn==testing_response).sum()
    print "Baseline (training): ",  np.sum(training_response==True) / float(len(training_response))
    print "Percent correct (training): ", (the_model.predict(training_predictors)==
                                           training_response).sum() / float(len(training_response))
    print "Percent correct (testing): ", (classfn==testing_response).sum() / float(len(classfn))
    
    print sklearn.metrics.confusion_matrix(testing_response.astype(int), classfn.astype(int))
    print "Precision: ", sklearn.metrics.precision_score(testing_response.astype(int), classfn.astype(int))
    print "Recall: ", sklearn.metrics.recall_score(testing_response.astype(int), classfn.astype(int))
    
    return the_model


#clf.fit(training_predictors, training[:,0])
#clf
for cval in (100000,):
    print("C =", cval)
    sample_rows = range(0,training_predictors.shape[0])
    svm = try_model(training_predictors[sample_rows,:], training.iloc[sample_rows,0], testing_predictors, testing.iloc[:,0], SVC(C=cval,class_weight='auto'))
    joblib.dump(svm, 'svm_{}.pkl'.format(int(time.time())))
    